- Cada dia é dividido em três turnos de oito horas.
- Todos os dias, cada turno é atribuído a um único enfermeiro, e nenhum profissional trabalha mais mais de um turno.
- Cada enfermeiro recebe pelo menos dois turnos durante o período de três dias.

In [ ]:
import random

def create_initial_solution():
    """Gera uma solução inicial aleatória para o agendamento de turnos."""
    return [[random.randint(0, n_enfermeiras - 1) for _ in range(n_turnos)] for _ in range(n_dias)]

def create_hotstart_solution():
    """Gera uma solução inicial usando uma abordagem de "hotstart" baseada nos turnos requisitados."""
    solution = [[-1 for _ in range(n_turnos)] for _ in range(n_dias)]
    for d in range(n_dias):
        for s in range(n_turnos):
            possible_nurses = [n for n in range(n_enfermeiras) if turnos_requisitados[n][d][s] == 1]
            solution[d][s] = random.choice(possible_nurses) if possible_nurses else random.randint(0, n_enfermeiras - 1)
    return solution

def calculate_fitness(solution):
    """Calcula a aptidão de uma solução com base nos critérios estabelecidos."""
    fitness = 0
    nurse_shifts = {n: 0 for n in range(n_enfermeiras)}
    
    for d in range(n_dias):
        for s in range(n_turnos):
            nurse = solution[d][s]
            nurse_shifts[nurse] += 1
            
            # Critérios de aptidão
            if turnos_requisitados[nurse][d][s] == 1:
                fitness += 2  # Preferência atendida
            if nurse_shifts[nurse] > max_turnos_semanais:
                fitness -= 5  # Penalidade por excesso de turnos
            if s == 2 and d < n_dias - 1 and solution[d + 1][0] == nurse:
                fitness -= 3  # Penalidade por não ter descanso após turno noturno
    return fitness

def crossover(parent1, parent2):
    """Executa o crossover entre dois pais para gerar dois filhos."""
    child1, child2 = [], []
    for d in range(n_dias):
        child1.append([random.choice([p1, p2]) for p1, p2 in zip(parent1[d], parent2[d])])
        child2.append([random.choice([p1, p2]) for p1, p2 in zip(parent2[d], parent1[d])])
    return child1, child2

def mutate(solution, mutation_rate):
    """Aplica mutação na solução com uma determinada taxa de mutação."""
    if random.random() < mutation_rate:
        d, s = random.randint(0, n_dias - 1), random.randint(0, n_turnos - 1)
        solution[d][s] = random.randint(0, n_enfermeiras - 1)

def tournament_selection(population, fitness_values, tournament_size):
    """Seleciona um indivíduo da população usando seleção por torneio."""
    selected = random.sample(list(zip(population, fitness_values)), tournament_size)
    return max(selected, key=lambda x: x[1])[0]

def create_initial_population(population_size, num_hotstart):
    """Cria a população inicial combinando soluções aleatórias e hotstart."""
    return [create_hotstart_solution() for _ in range(num_hotstart)] + [create_initial_solution() for _ in range(population_size - num_hotstart)]

def generate_new_population(population, fitness_values, population_size, mutation_rate, tournament_size):
    """Gera uma nova população através de crossover e mutação."""
    new_population = []
    best_solution = max(zip(population, fitness_values), key=lambda x: x[1])[0]
    new_population.append(best_solution)
    
    while len(new_population) < population_size:
        parent1 = tournament_selection(population, fitness_values, tournament_size)
        parent2 = tournament_selection(population, fitness_values, tournament_size)
        child1, child2 = crossover(parent1, parent2)
        mutate(child1, mutation_rate)
        mutate(child2, mutation_rate)
        new_population.extend([child1, child2])
        if len(new_population) >= population_size:
            break
    
    return new_population[:population_size]

def genetic_algorithm(population_size, mutation_rate, max_no_improvement_generations):
    """Executa o algoritmo genético até a convergência ou o limite de gerações sem melhoria."""
    num_hotstart = int(population_size * 0.1)
    population = create_initial_population(population_size, num_hotstart)
    
    fitness_history = []
    no_improvement_counter = 0
    best_fitness_so_far = float('-inf')
    
    generation = 0
    while no_improvement_counter < max_no_improvement_generations:
        fitness_values = [calculate_fitness(solution) for solution in population]
        best_solution = max(zip(population, fitness_values), key=lambda x: x[1])[0]
        best_fitness = calculate_fitness(best_solution)
        
        fitness_history.append(best_fitness)
        print(f"Geração {generation + 1}: Aptidão = {best_fitness}")
        
        # Verifique se a aptidão melhorou
        if best_fitness > best_fitness_so_far:
            best_fitness_so_far = best_fitness
            no_improvement_counter = 0  # Reset counter if improvement
        else:
            no_improvement_counter += 1
        
        # Geração da próxima população
        population = generate_new_population(population, fitness_values, population_size, mutation_rate, tamanho_torneio)
        generation += 1
    
    # Retorna a melhor solução encontrada
    best_solution = max(population, key=calculate_fitness)
    return best_solution, calculate_fitness(best_solution)

def generate_shift_requests(num_nurses, num_days, num_shifts):
    """Gera uma matriz de requisição de turnos para as enfermeiras."""
    return [[[random.choice([0, 1]) for _ in range(num_shifts)] for _ in range(num_days)] for _ in range(num_nurses)]

# Defina as variáveis globais
n_enfermeiras = 11
n_turnos = 3
max_turnos_semanais = 7
n_dias = 180
n_convergencia = 100  # Critério de parada por convergência
population_size = 500
mutation_rate = 0.15
tamanho_torneio = 5


# Geração dos turnos requisitados para as enfermeiras
turnos_requisitados = generate_shift_requests(n_enfermeiras, n_dias, n_turnos)

# Chamada da função com o critério de parada por convergência
best_solution, best_fitness = genetic_algorithm(population_size, mutation_rate, n_convergencia)

# Exibição do resultado final
print("Melhor solução encontrada:")
for d in range(n_dias):
    print(f"Dia {d}: {best_solution[d]}")
print(f"Aptidão da melhor solução: {best_fitness}")
